In [1]:
from pathlib import Path
import numpy as np
import tensorflow as tf
from ResNet import *
from InceptionNet import *

from tensorflow.keras import layers
from tensorflow.keras import Model
np.random.seed(42)
tf.keras.utils.set_random_seed(42)

In [2]:
featdir = Path('audio_features')
embdir = Path('embeddings')
weightsdir = Path('weights')
lbldir = Path('labels')
datadir = Path('datadir')

In [3]:
NClass = 49

SMALL = 0
shape = shape = (64, 64, 3) if SMALL else (160, 160, 3)

In [4]:
class dataHolder():
  def __init__(self):
    self.prepareData()
                      
  def prepareData(self):
    ADD = '_64' if SMALL else '_160'
    # ~ 5 gigs RAM overall
    self.trainimg = np.load(datadir / f'train_faces_only{ADD}.npy')
    self.valimg   = np.load(datadir / f'val_faces_only{ADD}.npy')
    self.testimg  = np.load(datadir / f'test_faces_only{ADD}.npy')

In [5]:
trainlabels = np.load(lbldir / 'tr_lbl.npy')
vallabels   = np.load(lbldir / 'val_lbl.npy')
testlabels  = np.load(lbldir / 'tst_lbl.npy')

train_out = tf.keras.utils.to_categorical(trainlabels, NClass)
val_out   = tf.keras.utils.to_categorical(vallabels, NClass)
test_out  = tf.keras.utils.to_categorical(testlabels, NClass)

## Load our Encoder

In [6]:
def get_encoder(Net):

  if Net==InceptionResNetV2:
    model = Net(include_top=1,
                weights=None,
                input_tensor=None,
                input_shape=shape,
                pooling=None,
                classes=49,
                classifier_activation="softmax")

  elif Net==ResNet50 or Net==ResNet_small:
    model = Net(include_top=1,
                weights=None,#"imagenet",
                input_tensor=None,
                input_shape=shape,
                pooling=None,
                classes=49)

  # add some dropout at the end:
  # (this was used in the training, and is not useful here.
  #  but we keep it for consistency)
  l2 = model.layers[-3]
  l1 = model.layers[-2]
  output = model.layers[-1]

  dropout1 = tf.keras.layers.Dropout(0.7)
  dropout2 = tf.keras.layers.Dropout(0.5)

  x = dropout1(l2.output)
  x = l1(x)
  x = dropout2(x)
  outputdrop = output(x)

  modeldrop = Model(inputs=model.input, outputs=outputdrop)

  return modeldrop

## Compute Embeddings

In [7]:
RECOMPUTE_EMBEDDINGS = True #False
if RECOMPUTE_EMBEDDINGS:
    model = get_encoder(InceptionResNetV2)

    # load the weights:
    model.load_weights(weightsdir / '28epochscrosscat_30epochstriplet_inceptionresnet_smaller_sepconv.h5')    
    
    data = dataHolder()
    
    # make the embeddings:
    embtest = model.predict(data.testimg)
    embtrain = model.predict(data.trainimg)
    embval = model.predict(data.valimg)
    
    np.save(embdir / 'embeddings_val_img.npy',   embval)
    np.save(embdir / 'embeddings_test_img.npy' , embtest)
    np.save(embdir / 'embeddings_train_img.npy', embtrain)

else:
    embval = np.load(embdir / 'embeddings_val_img.npy')
    embtest = np.load(embdir / 'embeddings_test_img.npy')
    embtrain = np.load(embdir / 'embeddings_train_img.npy')

2022-05-31 14:28:08.669580: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


30/30 [==============================] - 8s 260ms/step


## Classify Images Alone

In [8]:
# now we experiment and classify the embeddings of images alone with a 
# dense net.

def get_classifier():
    input_shape = (49)
    inputs = layers.Input(shape=input_shape)

    layer2 = layers.Dense(49)(inputs)
    outputs = layers.Activation('softmax')(layer2)

    model_classifier = Model(inputs=inputs, outputs=outputs)

    return model_classifier

classifier = get_classifier()
print(classifier.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 49)]              0         
                                                                 
 dense (Dense)               (None, 49)                2450      
                                                                 
 activation_44 (Activation)  (None, 49)                0         
                                                                 
Total params: 2,450
Trainable params: 2,450
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
loss = tf.keras.losses.CategoricalCrossentropy()
trainclass = True #False
if trainclass:

    # Compile the model
    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(0.1),
        loss=loss,
        metrics=['accuracy'])
    
    history = classifier.fit(
        embtrain, train_out,
        validation_data=(embval, val_out),
        batch_size=3000,
        epochs=150,
        shuffle=True)
    

else:
    classifier.load_weights(weightsdir / 'image_classifier.h5')

Epoch 1/150
1/1 [==============================] - 1s 629ms/step - loss: 3.9143 - accuracy: 0.0807 - val_loss: 3.7111 - val_accuracy: 0.1547
Epoch 2/150
1/1 [==============================] - 0s 28ms/step - loss: 3.6822 - accuracy: 0.1982 - val_loss: 3.5095 - val_accuracy: 0.4916
Epoch 3/150
1/1 [==============================] - 0s 28ms/step - loss: 3.4598 - accuracy: 0.5505 - val_loss: 3.3171 - val_accuracy: 0.5937
Epoch 4/150
1/1 [==============================] - 0s 27ms/step - loss: 3.2473 - accuracy: 0.6593 - val_loss: 3.1333 - val_accuracy: 0.6284
Epoch 5/150
1/1 [==============================] - 0s 28ms/step - loss: 3.0437 - accuracy: 0.6972 - val_loss: 2.9570 - val_accuracy: 0.6484
Epoch 6/150
1/1 [==============================] - 0s 24ms/step - loss: 2.8478 - accuracy: 0.7295 - val_loss: 2.7873 - val_accuracy: 0.6600
Epoch 7/150
1/1 [==============================] - 0s 24ms/step - loss: 2.6586 - accuracy: 0.7407 - val_loss: 2.6232 - val_accuracy: 0.6947
Epoch 8/150
1/1 [==

## Testing

In [10]:
pred = np.argmax(classifier(embtest), axis=1)
print(f"Test accuracy : {np.sum(pred==testlabels)/pred.size*100:.01f}%")

Test accuracy : 93.2%
